# 语义分块

根据语义相似性拆分文本。

摘自格雷格·卡姆拉特（Greg Kamradt）的精彩笔记本：[5_Levels_Of_Text_Splitting](https://github.com/FullStackRetrieval-com/RetrievalTutorials/blob/main/tutorials/LevelsOfTextSplitting/5_Levels_Of_Text_Splitting.ipynb)
所有的功劳都归功于他。

在高层次上，这会拆分为句子，然后分组成 3 个句子的组，然后合并嵌入空间中相似的一个句子。

安装依赖项

```
pip install --quiet langchain_experimental langchain_openai
```

加载示例数据

In [2]:
# This is a long document we can split up.
with open("./files/liudehua.txt") as f:
    liudehua = f.read()

创建文本拆分器

In [3]:
from langchain_experimental.text_splitter import SemanticChunker
from langchain_openai.embeddings import OpenAIEmbeddings

In [15]:
from langchain_cohere import CohereEmbeddings

import getpass
import os

# text_splitter = SemanticChunker(OpenAIEmbeddings(openai_api_key="EMPTY", openai_api_base="http://localhost:8000/v1", model= "text-embedding-ada-002"))
os.environ["COHERE_API_KEY"] = getpass.getpass()
embeddings = CohereEmbeddings(model="embed-multilingual-v3.0")

text_splitter = SemanticChunker(embeddings)

拆分文本

In [16]:
docs = text_splitter.create_documents([liudehua])
print(docs[0].page_content)

刘德华，BBS，MH，JP（英语：Andy Lau Tak Wah；1961年9月27日—），香港男演员、歌手、填词人、监制及出品人，1990年代获封为香港乐坛“四大天王”之一[3]，也是吉尼斯世界纪录大全中获奖最多的香港歌手[4]；在影视方面，他三次获得香港电影金像奖最佳男主角奖，两次获得金马奖最佳男主角奖，至今参演电影超过170部[5]。刘德华是天幕公司和映艺集团的创建者，作为投资人与监制已参与制作了30多部华语电影[6]。除此之外，刘德华是四川省川剧学校客座教授[7]。

1999年，刘德华获得“香港十大杰出青年”的荣誉，2000年11月则顺利荣登“世界十大杰出青年”[8]，成为获此殊荣的少数几位香港艺人。2006年7月7日，香港演艺学院因他“是香港最受尊重和喜爱的演艺名人之一，对香港电影及音乐贡献良多。其严谨专业的工作态度，足以成为年轻人的典范”，为了“表彰他在表演艺术方面的成就”而授予刘德华荣誉院士称号[9]，他也因此成为少数几位获此荣誉的香港艺人之一[10]。

刘德华笃信佛教，法号“慧果”，热心公益，时常参与慈善活动。2008年，刘德华获香港特别行政区政府委任为太平绅士[11]，2010年4月23日，刘德华获任中国残疾人福利基金会理事并担任副理事长[12]。2010年5月2日，刘德华获颁第十二届“世界杰出华人奖”同时获颁授加拿大纽奔驰域蓝仕桥大学荣誉博士学位[13]。2013年12月8日，他又当选香港残疾人奥委会暨伤残人士体育协会副会长[14]。2017年12月他因演艺事业和公益事务上的成就被香港树仁大学授予荣誉文学博士学位。2018年6月受邀加入美国电影艺术与科学学会成为会员[15]。


刘德华出生于香港新界大埔泰亨村（旧称菜坑村[注 1]）[16]，籍贯广东新会县荷塘镇（今江门市）[17]。祖父在当时算是大地主（乡村、农地）[18]。另外，他在家中亦有三姊、一妹和一弟（刘德盛）[19][20][21]，自己在家中排行第四[22]。其父刘礼年轻时为启德机场的消防员[16]。

刘德华五岁多时由于任职驻守机场消防员的父亲希望他能入读英文小学而随家人离开了农村[注 1]，全家后来搬到了九龙钻石山大磡村（曾就读村里的大磡村街坊福利会小学）[23]。钻石山为贫民区，多是木屋，容易发生火灾。刘家的木屋在刘德华十一岁时被大火烧毁，家人因此住在寮屋一年[18]，

断点

这个分块器的工作原理是确定何时“分解”句子。这是通过寻找任意两个句子之间的嵌入差异来完成的。当该差异超过某个阈值时，它们就会被拆分。
有几种方法可以确定该阈值是多少。

百分比
默认的拆分方式是基于百分位数。在这种方法中，计算句子之间的所有差异，然后拆分任何大于 X 百分位数的差异。

In [17]:
# text_splitter = SemanticChunker(
#     OpenAIEmbeddings(), breakpoint_threshold_type="percentile"
# )
text_splitter = SemanticChunker(
    embeddings, breakpoint_threshold_type="percentile"
)

```
    "percentile": 95,
    "standard_deviation": 3,
    "interquartile": 1.5,
```

In [18]:
docs = text_splitter.create_documents([liudehua])
print(docs[0].page_content)

刘德华，BBS，MH，JP（英语：Andy Lau Tak Wah；1961年9月27日—），香港男演员、歌手、填词人、监制及出品人，1990年代获封为香港乐坛“四大天王”之一[3]，也是吉尼斯世界纪录大全中获奖最多的香港歌手[4]；在影视方面，他三次获得香港电影金像奖最佳男主角奖，两次获得金马奖最佳男主角奖，至今参演电影超过170部[5]。刘德华是天幕公司和映艺集团的创建者，作为投资人与监制已参与制作了30多部华语电影[6]。除此之外，刘德华是四川省川剧学校客座教授[7]。

1999年，刘德华获得“香港十大杰出青年”的荣誉，2000年11月则顺利荣登“世界十大杰出青年”[8]，成为获此殊荣的少数几位香港艺人。2006年7月7日，香港演艺学院因他“是香港最受尊重和喜爱的演艺名人之一，对香港电影及音乐贡献良多。其严谨专业的工作态度，足以成为年轻人的典范”，为了“表彰他在表演艺术方面的成就”而授予刘德华荣誉院士称号[9]，他也因此成为少数几位获此荣誉的香港艺人之一[10]。

刘德华笃信佛教，法号“慧果”，热心公益，时常参与慈善活动。2008年，刘德华获香港特别行政区政府委任为太平绅士[11]，2010年4月23日，刘德华获任中国残疾人福利基金会理事并担任副理事长[12]。2010年5月2日，刘德华获颁第十二届“世界杰出华人奖”同时获颁授加拿大纽奔驰域蓝仕桥大学荣誉博士学位[13]。2013年12月8日，他又当选香港残疾人奥委会暨伤残人士体育协会副会长[14]。2017年12月他因演艺事业和公益事务上的成就被香港树仁大学授予荣誉文学博士学位。2018年6月受邀加入美国电影艺术与科学学会成为会员[15]。


刘德华出生于香港新界大埔泰亨村（旧称菜坑村[注 1]）[16]，籍贯广东新会县荷塘镇（今江门市）[17]。祖父在当时算是大地主（乡村、农地）[18]。另外，他在家中亦有三姊、一妹和一弟（刘德盛）[19][20][21]，自己在家中排行第四[22]。其父刘礼年轻时为启德机场的消防员[16]。

刘德华五岁多时由于任职驻守机场消防员的父亲希望他能入读英文小学而随家人离开了农村[注 1]，全家后来搬到了九龙钻石山大磡村（曾就读村里的大磡村街坊福利会小学）[23]。钻石山为贫民区，多是木屋，容易发生火灾。刘家的木屋在刘德华十一岁时被大火烧毁，家人因此住在寮屋一年[18]，

In [19]:
print(len(docs))

1


标准差
在这种方法中，任何大于 X 标准差的差异都会被拆分。

In [9]:
# text_splitter = SemanticChunker(
#     OpenAIEmbeddings(openai_api_key="EMPTY", openai_api_base="http://localhost:8000/v1", model= "text-embedding-ada-002"), breakpoint_threshold_type="standard_deviation"
# )
text_splitter = SemanticChunker(
    embeddings, breakpoint_threshold_type="standard_deviation"
)

In [10]:
docs = text_splitter.create_documents([liudehua])
print(docs[0].page_content)

刘德华，BBS，MH，JP（英语：Andy Lau Tak Wah；1961年9月27日—），香港男演员、歌手、填词人、监制及出品人，1990年代获封为香港乐坛“四大天王”之一[3]，也是吉尼斯世界纪录大全中获奖最多的香港歌手[4]；在影视方面，他三次获得香港电影金像奖最佳男主角奖，两次获得金马奖最佳男主角奖，至今参演电影超过170部[5]。刘德华是天幕公司和映艺集团的创建者，作为投资人与监制已参与制作了30多部华语电影[6]。除此之外，刘德华是四川省川剧学校客座教授[7]。

1999年，刘德华获得“香港十大杰出青年”的荣誉，2000年11月则顺利荣登“世界十大杰出青年”[8]，成为获此殊荣的少数几位香港艺人。2006年7月7日，香港演艺学院因他“是香港最受尊重和喜爱的演艺名人之一，对香港电影及音乐贡献良多。其严谨专业的工作态度，足以成为年轻人的典范”，为了“表彰他在表演艺术方面的成就”而授予刘德华荣誉院士称号[9]，他也因此成为少数几位获此荣誉的香港艺人之一[10]。

刘德华笃信佛教，法号“慧果”，热心公益，时常参与慈善活动。2008年，刘德华获香港特别行政区政府委任为太平绅士[11]，2010年4月23日，刘德华获任中国残疾人福利基金会理事并担任副理事长[12]。2010年5月2日，刘德华获颁第十二届“世界杰出华人奖”同时获颁授加拿大纽奔驰域蓝仕桥大学荣誉博士学位[13]。2013年12月8日，他又当选香港残疾人奥委会暨伤残人士体育协会副会长[14]。2017年12月他因演艺事业和公益事务上的成就被香港树仁大学授予荣誉文学博士学位。2018年6月受邀加入美国电影艺术与科学学会成为会员[15]。


刘德华出生于香港新界大埔泰亨村（旧称菜坑村[注 1]）[16]，籍贯广东新会县荷塘镇（今江门市）[17]。祖父在当时算是大地主（乡村、农地）[18]。另外，他在家中亦有三姊、一妹和一弟（刘德盛）[19][20][21]，自己在家中排行第四[22]。其父刘礼年轻时为启德机场的消防员[16]。

刘德华五岁多时由于任职驻守机场消防员的父亲希望他能入读英文小学而随家人离开了农村[注 1]，全家后来搬到了九龙钻石山大磡村（曾就读村里的大磡村街坊福利会小学）[23]。钻石山为贫民区，多是木屋，容易发生火灾。刘家的木屋在刘德华十一岁时被大火烧毁，家人因此住在寮屋一年[18]，

In [11]:
print(len(docs))

1


四分位数
在这种方法中，四分位距用于分割块。

In [12]:
# text_splitter = SemanticChunker(
#     OpenAIEmbeddings(openai_api_key="EMPTY", openai_api_base="http://localhost:8000/v1", model= "text-embedding-ada-002"), breakpoint_threshold_type="interquartile"
# )
text_splitter = SemanticChunker(
    embeddings, breakpoint_threshold_type="interquartile"
)

In [13]:
docs = text_splitter.create_documents([liudehua])
print(docs[0].page_content)

刘德华，BBS，MH，JP（英语：Andy Lau Tak Wah；1961年9月27日—），香港男演员、歌手、填词人、监制及出品人，1990年代获封为香港乐坛“四大天王”之一[3]，也是吉尼斯世界纪录大全中获奖最多的香港歌手[4]；在影视方面，他三次获得香港电影金像奖最佳男主角奖，两次获得金马奖最佳男主角奖，至今参演电影超过170部[5]。刘德华是天幕公司和映艺集团的创建者，作为投资人与监制已参与制作了30多部华语电影[6]。除此之外，刘德华是四川省川剧学校客座教授[7]。

1999年，刘德华获得“香港十大杰出青年”的荣誉，2000年11月则顺利荣登“世界十大杰出青年”[8]，成为获此殊荣的少数几位香港艺人。2006年7月7日，香港演艺学院因他“是香港最受尊重和喜爱的演艺名人之一，对香港电影及音乐贡献良多。其严谨专业的工作态度，足以成为年轻人的典范”，为了“表彰他在表演艺术方面的成就”而授予刘德华荣誉院士称号[9]，他也因此成为少数几位获此荣誉的香港艺人之一[10]。

刘德华笃信佛教，法号“慧果”，热心公益，时常参与慈善活动。2008年，刘德华获香港特别行政区政府委任为太平绅士[11]，2010年4月23日，刘德华获任中国残疾人福利基金会理事并担任副理事长[12]。2010年5月2日，刘德华获颁第十二届“世界杰出华人奖”同时获颁授加拿大纽奔驰域蓝仕桥大学荣誉博士学位[13]。2013年12月8日，他又当选香港残疾人奥委会暨伤残人士体育协会副会长[14]。2017年12月他因演艺事业和公益事务上的成就被香港树仁大学授予荣誉文学博士学位。2018年6月受邀加入美国电影艺术与科学学会成为会员[15]。


刘德华出生于香港新界大埔泰亨村（旧称菜坑村[注 1]）[16]，籍贯广东新会县荷塘镇（今江门市）[17]。祖父在当时算是大地主（乡村、农地）[18]。另外，他在家中亦有三姊、一妹和一弟（刘德盛）[19][20][21]，自己在家中排行第四[22]。其父刘礼年轻时为启德机场的消防员[16]。

刘德华五岁多时由于任职驻守机场消防员的父亲希望他能入读英文小学而随家人离开了农村[注 1]，全家后来搬到了九龙钻石山大磡村（曾就读村里的大磡村街坊福利会小学）[23]。钻石山为贫民区，多是木屋，容易发生火灾。刘家的木屋在刘德华十一岁时被大火烧毁，家人因此住在寮屋一年[18]，

In [14]:
print(len(docs))

1
